<a href="https://colab.research.google.com/github/natsumiA/Fashion_MNIST/blob/main/keras_randomsearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.wrappers.scikit_learn import KerasClassifier
from keras import backend as K
from sklearn.model_selection import RandomizedSearchCV
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import KFold
from tensorflow.keras import optimizers

In [10]:
num_classes = 10
img_rows, img_cols = 28, 28

In [11]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [12]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test .reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [13]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /=255

In [14]:
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

In [17]:
def make_model(num_layer, activation, mid_units, num_filters, dropout_rate, optimizer, learning_rate):

    model = Sequential()
    model.add(Conv2D(int(num_filters[0]), (3,3), padding='same', input_shape=input_shape))
    model.add(Activation(activation))
    '''
    for i in range(1, len(num_filters), num_layer):
        model.add(Conv2D(int(num_filters[i]), (3,3)))
        model.add(Activation(activation))
    '''    

    model.add(Conv2D(16, (3,3)))
    model.add(Activation(activation))
    
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(dropout_rate))
    
    model.add(Flatten())
    
    model.add(Dense(mid_units))
    model.add(Activation(activation))
    model.add(Dense(10))
    model.add(Activation('softmax'))
    
    model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

    if optimizer == "sgd":
        opt = optimizers.SGD(learning_rate)
    elif optimizer == "adam":
        opt = optimizers.Adam(learning_rate)
    elif optimizer == "RMSprop":
        opt = optimizers.rmsprop(learning_rate)


    return model

In [18]:
my_classifier = KerasClassifier(make_model, batch_size=128)

validator = RandomizedSearchCV(my_classifier,
                         param_distributions={
                                     'num_layer' : [1,2,3,4,5],
                                     'mid_units' : [32, 64, 128] ,
                                     'num_filters' : ['16', '32','128', '32', '16'],
                                     'activation' : ['relu', 'sigmoid'],
                                     'dropout_rate' : [0.3, 0.5],
                                     'optimizer':['sgd', 'adam', 'rmsprop'],
                                     'learning_rate' : [1e-7, 1e0]},
                         scoring="neg_log_loss",
                         error_score='raise', 
                         n_jobs=1)

validator.fit(x_train, y_train,epochs=30,validation_data=(x_test, y_test))

print('The parameters of the best model are: ')
print(validator.best_params_)
print(validator.best_score_)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  """Entry point for launching an IPython kernel.


Epoch 1/30
375/375 [==============================] - 13s 11ms/step - loss: 1.5636 - accuracy: 0.4388 - val_loss: 0.7405 - val_accuracy: 0.7402
Epoch 2/30
375/375 [==============================] - 4s 10ms/step - loss: 0.6588 - accuracy: 0.7602 - val_loss: 0.5841 - val_accuracy: 0.7883
Epoch 3/30
375/375 [==============================] - 4s 11ms/step - loss: 0.5622 - accuracy: 0.7948 - val_loss: 0.5237 - val_accuracy: 0.8076
Epoch 4/30
375/375 [==============================] - 4s 12ms/step - loss: 0.5113 - accuracy: 0.8125 - val_loss: 0.4839 - val_accuracy: 0.8246
Epoch 5/30
375/375 [==============================] - 4s 12ms/step - loss: 0.4762 - accuracy: 0.8263 - val_loss: 0.4511 - val_accuracy: 0.8334
Epoch 6/30
375/375 [==============================] - 4s 11ms/step - loss: 0.4458 - accuracy: 0.8362 - val_loss: 0.4286 - val_accuracy: 0.8473
Epoch 7/30
375/375 [==============================] - 4s 10ms/step - loss: 0.4227 - accuracy: 0.8445 - val_loss: 0.4066 - val_accuracy: 0.855